In [ ]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [ ]:
from lib import *

In [ ]:
wdir = "/media/windows/projects/icon-evaluation/kriging/"

variables = ['precipitation', 'temperature', 'streamflow']

basin = "passirio"

start_date_str = '2010-01-01 00:00:00'
end_date_str = '2021-10-17 00:00:00'
timezone_str = 'Europe/Rome'

start_date = dt.datetime.strptime(start_date_str, '%Y-%m-%d %H:%M:%S')
end_date = dt.datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')

output_dt_format = '%Y-%m-%dT%H:%M:%S.%f%z'

In [ ]:
output_weather_data_path = wdir + "data/passirio/GS/"

### stations metadata
metadata_data_path =  wdir + "data/AA_weather_data/AA_stations_metadata.csv"

metadata = pd.read_csv( metadata_data_path )
metadata = metadata[ metadata['ID_UI'] != '-999' ]

In [ ]:
original_weather_data_path = "/media/windows/data/meteo/eu/it/taa/aa/weather_stations/merged/"
input_dt_format = '%Y-%m-%dT%H:%M:%S.%f%z'

##### A. Creation of input for the kriging

- for each variable to create the grid data set
    - for each station available in the input directory
        - select metadata of the station
        - read the data of each available station data [ only the currently updated == the stations with ID_UI ]
        - write the metadata and the data in the output directory in the format of kriging input

In [ ]:
for variable in variables:

    st_list = []
    for m in metadata.index:

        internal_id = str(metadata.loc[m]['ID'])
        # print(internal_id)

        east = str(metadata.loc[m]['East'])
        north = str(metadata.loc[m]['North'])
        elevation = str(metadata.loc[m]['Elevation'])
        id_ui = str(metadata.loc[m]['ID_UI'])

        current_data_path = original_weather_data_path + \
            variable + "/" + str(internal_id) + ".csv"
        # print(current_data_path)

        try:
            current_data = read_timeseries_pd( pd.read_csv(current_data_path, header=0), input_dt_format=input_dt_format )
        except FileNotFoundError as err:
            print("FileNotFoundError: {0}".format(err))
            continue
        except BaseException as err:
            print(f"Unexpected {err=}, {type(err)=}")
            raise

        if variable == 'precipitation':
            current_data = current_data.resample('h').sum()
        elif variable == 'temperature':
            current_data = current_data.resample('h').mean()
        elif variable == 'streamflow':
            current_data = current_data.resample('h').mean()
        else:
            print('NOT A VALID VARIABLE!')
            continue
            
        current_data.index = [dt.datetime.strftime(
            i, format=output_dt_format) for i in current_data.index]

        df = current_data.to_csv(header=False).strip('\n').split('\n')
        # <= this is the string that you can use with md5
        data = '\r\n'.join(df)
        # data = df_string.encode('utf8')  # <= this is bytes object to write the file
        # print(df_bytes)

        current_output_file = output_weather_data_path + \
            variable + "/" + internal_id + ".txt"
        mkNestedDir(getPathFromFilepath(current_output_file))

        # ID,2.0
        # x,642993.5
        # y,5164882.0
        # z,630.0
        header = """ID,{id}\nx,{x}\ny,{y}\nz,{z}\n"""

        with open(current_output_file, 'w') as new:
            new.write(header.format(id=internal_id,
                      x=east, y=north, z=elevation))
            new.write(data)

        # add station to station list
        st_props = {}

        st_props["station_id"] = internal_id
        st_props["station_name"] = str(metadata.loc[m]['Station']).lower().replace(
            " ", "_").replace("-", "_").replace("(", "_").replace(")", "_")
        st_props["east"] = east
        st_props["north"] = north
        st_props["elevation"] = elevation

        st_list.append(st_props)

    with open(output_weather_data_path + variable + ".json", "w") as data_file:
        json.dump(st_list, data_file, indent=4, sort_keys=True)

        # print(current_output_file)

##### Creation of the *{variable}*.json metadata

**NB:** BEFORE is necessary to create the file output from the extracted data and local DTM - the same of forecasting data

In [ ]:
grid_file = wdir + "data/" + basin + "/output.csv"
grid_metadata = pd.read_csv( grid_file, index_col=1 )
# grid_metadata.drop(columns=["Unnamed: 0"], inplace=True)

grid_metadata.dropna(inplace=True)

st_list = []

for id in grid_metadata.index:

    st_props = {}

    st_props["station_id"] = str(id)
    st_props["station_name"] = str(id)
    st_props["east"] = str(grid_metadata.loc[id]['lat'])
    st_props["north"] = str(grid_metadata.loc[id]['lon'])
    st_props["elevation"] = str(grid_metadata.loc[id]['elevation'])

    st_list.append( st_props )

with open( wdir + "data/" + basin + "/grid.json", "w") as data_file:
    json.dump(st_list, data_file, indent=4, sort_keys=True)